In [1]:
import cv2
import numpy as np
import imutils

In [2]:
def show(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return

In [3]:
def grid_ext(image):
    # Grayscale conversion
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold
    _,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    thresh = cv2.dilate(thresh,None)
    thresh = ~thresh
    thresh = cv2.Canny(gray, 40, 200)
    
    # Closing
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 3))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    show("thresh", thresh)

    # ROI Detection
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)                      # need to understand imutils.grab_contours
    max_c = max(cnts, key=cv2.contourArea)                  # largest contour/ req grid
    cnt = [list(pt[0]) for pt in max_c]                     # converts into list of contour points

    # a = image.copy()
    # cv2.drawContours(a, c, -1, (0, 255, 255), 2)
    # cv2.imshow('cnt', a)
    # cv2.waitKey(0)

    b_r = max(cnt, key=lambda x: x[0] + x[1])               # b_r = bottom right
    t_l = min(cnt, key=lambda x: x[0] + x[1])               # t_l = top left
    t_r = max(cnt, key=lambda x: x[0] - x[1])               # t_r = top right
    b_l = min(cnt, key=lambda x: x[0] - x[1])               # b_l = bottom left

    # print(b_l, b_r, t_l, t_r)

#     b_r[0], b_r[1] = b_r[0] + 2, b_r[1] + 2
#     b_l[0], b_l[1] = b_l[0] - 2, b_l[1] + 2
#     t_r[0], t_r[1] = t_r[0] + 2, t_r[1] - 2
#     t_l[0], t_l[1] = t_l[0] - 2, t_l[1] - 2

    b_r[0], b_r[1] = b_r[0], b_r[1]
    b_l[0], b_l[1] = b_l[0], b_l[1]
    t_r[0], t_r[1] = t_r[0], t_r[1]
    t_l[0], t_l[1] = t_l[0], t_l[1]

    asp_rat = (b_l[1] - t_l[1]) / (t_r[0] - t_l[0])
    # print('aspect ratio:', asp_rat)

    # print('corners:', b_l, b_r, t_l, t_r)

    w = 600
    h = int(round(w*asp_rat))

    pts1 = np.float32([t_l, t_r, b_l, b_r])                 # pts1: from shape
    pts2 = np.float32([[0, 0], [h, 0], [0, w], [h, w]])     # pts2: to shape
    morph = cv2.getPerspectiveTransform(pts1, pts2)
    fin_image = cv2.warpPerspective(image, morph, (h, w))
    show('image',fin_image)
    return fin_image

In [4]:
def rem_multi_lines(lines, thresh):
    """
    to remove the multiple lines with close proximity 
    :param lines: initial list with all the lines(multiple in place of singular)
    :param thresh: dist between two lines for them to be considered as same
    :return: final list with singular lines in place of multiple
    """
    a = []
    i = 0
    lines.append([800, 0])                                  # random val/ noise
    out = []
    # this loop collects lines with close proximity in a list (a) and then appends that 
    # complete list in a common list called out.
    while i < len(lines) - 1:
        if lines[i] not in a:
            a.append(lines[i])
        if abs(lines[i + 1][0] - lines[i][0]) < thresh:
            a.append(lines[i + 1])
        else:
            out.append(a)
            a = []
        i += 1
    
    # print(out)

    final = []
    for i in out:
        a = np.array(i)
        final.append(np.average(a, axis=0))

    # print(final)

    for i in final.copy():
        if i[0] < 0:
            final.remove(i)
    return final


def draw_r_theta_lines(img, lines, color):
    """
    draw lines on image which are of (r, theta) form
    :param img: image to draw the lines on
    :param lines: list of lines on the form (r, theta)
    :param color: color of lines
    :return: 
    """
    for rho, theta in lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * a)

        cv2.line(img, (x1, y1), (x2, y2), color, 2)


def lines_ext(img, hough_thresh, multilines_thresh):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    
    show('image',edges)

    line_image = img.copy()

    lines = cv2.HoughLines(edges, 1, np.pi / 180, hough_thresh)
    lines = lines.reshape(lines.shape[0], 2)

    draw_r_theta_lines(line_image, lines, (0, 0, 255))

    lines = sorted(lines, key=lambda x: x[0])

    show("lines", line_image)

    l1 = list(lines)
    l2 = []
    for i in l1:
        l2.append(list(i))

    v_lines = []
    h_lines = []

    for i in l2:
        if round(i[1]) == 0:
            v_lines.append(i)
        elif round(i[1]) > 0.1:
            h_lines.append(i)

    # print('v:', v_lines)
    # print('h:', h_lines)

    v_lines = rem_multi_lines(v_lines, multilines_thresh)
    h_lines = rem_multi_lines(h_lines, multilines_thresh)

    final = v_lines + h_lines

    draw_r_theta_lines(line_image, final, (0, 255, 0))

    show("lines", line_image)

    return v_lines, h_lines


In [5]:
import numpy as np


def intersection_bw_2_lines(l1, l2):
    """
    Returns point of intersection between 2 lines
    Parameters:
        l1 : line1
        l2 : line2
    Returns:
        x and y coordinate of point of intersection of l1 and l2
    """
    rho1, theta1 = l1
    rho2, theta2 = l2

    a = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])

    x0, y0 = np.linalg.solve(a, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))

    return [x0, y0]


def cells_ext(h_lines, v_lines, start_r=0, start_c=0, end_r=0, end_c=0):
    """
    Extracts four corners of the cell from the horizontal and vertical lines POI
    :param h_lines: r, theta form horizontal lines
    :param v_lines: r, theta form vertical lines
    :param start_r: starting row number
    :param start_c: starting column number
    :param end_r: ending row number
    :param end_c: ending column number
    :return: final list containing the four corners of individual cells
    """
    if end_r == end_c == 0:
        end_c = len(v_lines) - 1
        end_r = len(h_lines) - 1
    ret_cell = []
    for i in range(0, len(h_lines) - 1):
        for j in range(0, len(v_lines) - 1):
            hl1, hl2 = h_lines[i], h_lines[i + 1]
            vl1, vl2 = v_lines[j], v_lines[j + 1]

            p1 = intersection_bw_2_lines(hl1, vl1)
            p2 = intersection_bw_2_lines(hl1, vl2)
            p3 = intersection_bw_2_lines(hl2, vl1)
            p4 = intersection_bw_2_lines(hl2, vl2)

            ret_cell.append([p1, p2, p3, p4])

    for i in range(len(ret_cell)):
        p1, p2, p3, p4 = ret_cell[i]
#         p1 = (p1[0] + 2, p1[1] + 2)
#         p2 = (p2[0] - 2, p2[1] + 2)
#         p3 = (p3[0] + 2, p3[1] - 2)
#         p4 = (p4[0] - 2, p4[1] - 2)
        p1 = (p1[0], p1[1])
        p2 = (p2[0], p2[1])
        p3 = (p3[0], p3[1])
        p4 = (p4[0], p4[1])
        ret_cell[i] = p1, p2, p3, p4

    ret_cell_fin = []

    for i in range(len(ret_cell)):
        if start_r <= (i / (len(v_lines) - 1)) <= (end_r + 1) and start_c <= (i % (len(v_lines) - 1)) <= end_c:
            ret_cell_fin.append(ret_cell[i])

    return ret_cell_fin


In [6]:
image = cv2.imread('/home/akshay/SEM6/IVP/cell extraction only/sample1.jpg')

show('original_img', cv2.resize(image, (950, 650)))

image = grid_ext(image)

v_lines, h_lines = lines_ext(image, 270, 5)

ret_cell_fin = cells_ext(h_lines, v_lines, 1, 2, 11, 7)

cell_image = image.copy()

for i in range(len(ret_cell_fin)):
    p1, p2, p3, p4 = ret_cell_fin[i]

    cell = cell_image[p1[1]:p3[1], p1[0]:p2[0]]
#     cv2.imwrite("cells1/" + str(i) + ".jpg", cell)
    show('cell',cell)